## Setup and Configuration

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import Crokinole as crk
from config import DEFAULT_CONFIG
from importlib import reload
reload(crk)

print("Libraries loaded successfully")

In [ ]:
# Configuration - loaded from config.py
CONFIG = DEFAULT_CONFIG.copy()

print("Configuration loaded")

## Load Dataset

In [ ]:
# Load imageset.csv
dataset_path = Path('images/imageset.csv')
df = pd.read_csv(dataset_path)

print(f"Loaded dataset with {len(df)} images")
print(f"Columns: {list(df.columns)}")
print("\nFirst few rows:")
df.head()

## Run Pipeline on All Images

In [ ]:
# Initialize results storage
results = []
images_base_path = Path('images')

# Process each image
for idx, row in df.iterrows():
    img_name = row['image_name']
    img_path = images_base_path / img_name
    
    gt_team1 = int(row['team1_score'])
    gt_team2 = int(row['team2_score'])
    team1_20s = int(row['team1_20s'])
    team2_20s = int(row['team2_20s'])
    
    print(f"[{idx+1}/{len(df)}] {img_name}... ", end="")
    
    # Run pipeline
    pipeline_result = crk.run_complete_pipeline(
        str(img_path),
        CONFIG,
        team1_20s=team1_20s,
        team2_20s=team2_20s
    )
    
    # Calculate relative percentage errors
    if pipeline_result['success']:
        pred_t1 = pipeline_result['team1_score']
        pred_t2 = pipeline_result['team2_score']
        
        # Absolute errors
        abs_error_team1 = abs(pred_t1 - gt_team1)
        abs_error_team2 = abs(pred_t2 - gt_team2)
        
        # Percentage error calculation:
        # If GT > 0: use relative error = |pred - gt| / gt * 100
        # If GT = 0: just use absolute error (can't compute meaningful %)
        error_pct_team1 = (abs_error_team1 / gt_team1 * 100) if gt_team1 > 0 else abs_error_team1
        error_pct_team2 = (abs_error_team2 / gt_team2 * 100) if gt_team2 > 0 else abs_error_team2
    else:
        error_pct_team1 = None
        error_pct_team2 = None
        abs_error_team1 = None
        abs_error_team2 = None
    
    # Store results
    result_entry = {
        'image_name': img_name,
        'success': pipeline_result['success'],
        'pred_team1': pipeline_result['team1_score'] if pipeline_result['success'] else None,
        'pred_team2': pipeline_result['team2_score'] if pipeline_result['success'] else None,
        'gt_team1': gt_team1,
        'gt_team2': gt_team2,
        'abs_error_team1': abs_error_team1,
        'abs_error_team2': abs_error_team2,
        'error_pct_team1': error_pct_team1,
        'error_pct_team2': error_pct_team2,
        'num_discs': len(pipeline_result['detected_discs']) if pipeline_result['detected_discs'] else 0,
        'error_msg': pipeline_result['error']
    }
    results.append(result_entry)
    
    # Store overlay image for later display
    result_entry['overlay_img'] = pipeline_result['overlay_img'] if pipeline_result['success'] else None
    
    # One-line result - format percentage or absolute based on GT
    if pipeline_result['success']:
        err_str_t1 = f"{error_pct_team1:.1f}%" if gt_team1 > 0 else f"{abs_error_team1} pts"
        err_str_t2 = f"{error_pct_team2:.1f}%" if gt_team2 > 0 else f"{abs_error_team2} pts"
        print(f"OK Pred: T0={pred_t1}, T1={pred_t2} | GT: T0={gt_team1}, T1={gt_team2} | Error: T0={err_str_t1}, T1={err_str_t2}")
    else:
        print(f"FAILED: {pipeline_result['error']}")

print(f"\nProcessing complete!")

## Display All Visualization Overlays

In [ ]:
# Display all overlay images with predictions and ground truth
print(f"\n{'='*80}")
print("VISUALIZATION OVERLAYS FOR ALL IMAGES")
print(f"{'='*80}\n")

for result in results:
    if result['success'] and result['overlay_img'] is not None:
        plt.figure(figsize=(10, 10))
        plt.imshow(result['overlay_img'])
        plt.axis('off')
        
        # Calculate if prediction was correct
        error_indicator = "✓ PERFECT" if result['total_error'] == 0 else f"Error: {result['total_error']}"
        
        title = f"{result['image_name']}\n"
        title += f"Predicted: T0={result['pred_team1']}, T1={result['pred_team2']} | "
        title += f"Ground Truth: T0={result['gt_team1']}, T1={result['gt_team2']}\n"
        title += f"{error_indicator}"
        
        plt.title(title, fontsize=12, fontweight='bold')
        plt.tight_layout()
        plt.show()
    elif not result['success']:
        print(f"{result['image_name']}: FAILED - {result['error_msg']}\n")

## Results Summary

In [ ]:
# Create results dataframe (excluding overlay images for CSV)
results_for_df = [{k: v for k, v in r.items() if k != 'overlay_img'} for r in results]
results_df = pd.DataFrame(results_for_df)

# Display full results table
print("\n" + "="*80)
print("COMPLETE RESULTS TABLE")
print("="*80)
print(results_df.to_string(index=False))

# Calculate statistics
successful = results_df[results_df['success'] == True]
num_success = len(successful)
num_total = len(results_df)

print(f"\n{'='*80}")
print("SUMMARY STATISTICS")
print(f"{'='*80}")
print(f"Total images processed: {num_total}")
print(f"Successful detections: {num_success} ({100*num_success/num_total:.1f}%)")
print(f"Failed detections: {num_total - num_success} ({100*(num_total-num_success)/num_total:.1f}%)")

if num_success > 0:
    print(f"\nScoring Accuracy (successful detections only):")
    print(f"  Mean percentage error (Team 0): {successful['error_pct_team1'].mean():.2f}%")
    print(f"  Mean percentage error (Team 1): {successful['error_pct_team2'].mean():.2f}%")
    print(f"  Mean absolute error (Team 0): {successful['abs_error_team1'].mean():.2f} points")
    print(f"  Mean absolute error (Team 1): {successful['abs_error_team2'].mean():.2f} points")
    perfect = successful[(successful['abs_error_team1'] == 0) & (successful['abs_error_team2'] == 0)]
    print(f"  Perfect predictions (both teams exact): {len(perfect)} ({100*len(perfect)/num_success:.1f}%)")
    print(f"  Mean discs detected: {successful['num_discs'].mean():.1f}")

# Save results to CSV
output_path = Path('results/testset_results.csv')
output_path.parent.mkdir(exist_ok=True)
results_df.to_csv(output_path, index=False)
print(f"\nResults saved to: {output_path}")

## Visualize Error Distribution

In [ ]:
# Plot per-image error distribution as dual bar graph
if num_success > 0:
    fig, ax = plt.subplots(figsize=(16, 6))
    
    # Get successful results sorted by image name
    plot_data = successful.sort_values('image_name')
    
    x = np.arange(len(plot_data))
    width = 0.35
    
    # Create bars
    bars1 = ax.bar(x - width/2, plot_data['error_pct_team1'], width, 
                   label='Team 0 Error %', color='skyblue', edgecolor='black')
    bars2 = ax.bar(x + width/2, plot_data['error_pct_team2'], width, 
                   label='Team 1 Error %', color='lightcoral', edgecolor='black')
    
    # Customize plot
    ax.set_xlabel('Images', fontsize=12, fontweight='bold')
    ax.set_ylabel('Percentage Error (%)', fontsize=12, fontweight='bold')
    ax.set_title('Per-Image Error Distribution (Percentage)', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(plot_data['image_name'], rotation=45, ha='right', fontsize=8)
    ax.legend(fontsize=10)
    ax.grid(axis='y', alpha=0.3)
    
    # Add mean lines
    mean_t1 = plot_data['error_pct_team1'].mean()
    mean_t2 = plot_data['error_pct_team2'].mean()
    ax.axhline(mean_t1, color='blue', linestyle='--', linewidth=1, alpha=0.7, 
               label=f'Team 0 Mean: {mean_t1:.1f}%')
    ax.axhline(mean_t2, color='red', linestyle='--', linewidth=1, alpha=0.7, 
               label=f'Team 1 Mean: {mean_t2:.1f}%')
    ax.legend(fontsize=10)
    
    plt.tight_layout()
    plt.show()
else:
    print("No successful detections to visualize")